## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

In [ ]:
!pip install ultralytics

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from ultralytics import YOLO, settings
import yaml
from keras.preprocessing import image

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os
from tqdm import tqdm

def dataset_extract(file_name):
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        file_list = zip_ref.namelist()

        # 압축을 해제할 폴더 경로 생성
        extract_folder = os.path.join(os.path.dirname(file_name), file_name[:-4])

        # 압축을 해제할 폴더가 이미 존재하는지 확인하고 없다면 생성
        if not os.path.exists(extract_folder):
            os.makedirs(extract_folder)
        else:
            print("데이터셋 폴더가 이미 존재합니다.")

        # 압축 해제 진행
        for f in tqdm(file_list, desc='Extracting', unit='files'):
            zip_ref.extract(member=f, path=extract_folder)

In [ ]:
# 새로운 폴더 생성 및 이미지 파일 이동
new_folder_path = os.path.join(path, 'cool_roof_images')
os.makedirs(new_folder_path, exist_ok=True)

new_label_path = os.path.join(path, 'cool_roof_yolo_labels')
os.makedirs(new_label_path, exist_ok=True)

In [ ]:
dataset_extract('/content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_images.zip')

데이터셋 폴더가 이미 존재합니다.


Extracting: 100%|██████████| 200/200 [00:02<00:00, 69.04files/s]


In [ ]:
dataset_extract('/content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_yolo_labels.zip')

데이터셋 폴더가 이미 존재합니다.


Extracting: 100%|██████████| 204/204 [00:01<00:00, 184.08files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [ ]:
def make_yolo_dirs(data_path):
    if os.path.exists(data_path):
        shutil.rmtree(data_path)

    if not os.path.exists(data_path + 'train'):
        os.makedirs(data_path + 'train/images')
        os.makedirs(data_path + 'train/labels')

    if not os.path.exists(data_path + 'valid'):
        os.makedirs(data_path + 'valid/images')
        os.makedirs(data_path + 'valid/labels')

    if not os.path.exists(data_path + 'test'):
        os.makedirs(data_path + 'test')

In [ ]:
data_path = '/content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/data/'
make_yolo_dirs(data_path)

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [ ]:
image_paths = glob.glob("/content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_images/*.jpg")
txt_paths = glob.glob("/content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/cool_roof_yolo_labels/obj_train_data/*.txt")

train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths = train_test_split(image_paths, txt_paths, test_size=0.2, random_state=2024)

In [ ]:
def file_split(train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths):
    # Train 이미지와 텍스트 파일을 이동
    for img_path, txt_path in zip(train_images_paths, train_txt_paths):
        shutil.move(img_path, data_path + 'train/images')
        shutil.move(txt_path, data_path + 'train/labels')

    # Valid 이미지와 텍스트 파일을 이동
    for img_path, txt_path in zip(valid_images_paths, valid_txt_paths):
        shutil.move(img_path, data_path + 'valid/images')
        shutil.move(txt_path, data_path + 'valid/labels')

In [ ]:
file_split(train_images_paths, valid_images_paths, train_txt_paths, valid_txt_paths)

In [ ]:
len(glob.glob(data_path + 'train/images/*jpg')), len(glob.glob(data_path + 'train/labels/*txt')), len(glob.glob(data_path + 'valid/images/*jpg')), len(glob.glob(data_path + 'valid/labels/*txt'))

(160, 160, 40, 40)

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [ ]:
data = {
    "names" : {0 : 'cool roof', 1 : 'generic roof'},
    "nc": 2,
    "path": data_path,
    "train": "train",
    "val": "valid",
    "test": "test",
    }

with open(data_path + 'data.yaml', 'w') as f :
    yaml.dump(data, f)

# # check written file
# with open(data_path + 'data.yaml', 'r') as f :
#     lines = yaml.safe_load(f)
#     print(lines)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
model = YOLO(model='yolov8s.pt', task='detect')

100%|██████████| 21.5M/21.5M [00:00<00:00, 296MB/s]


In [ ]:
model.train(data=data_path + 'data.yaml',
            epochs=200,
            patience=20,
            pretrained=True,
            verbose=True,
            seed=2024,
            )

Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/data/data.yaml, epochs=200, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

train: Scanning /content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/data/train/labels... 160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<00:00, 164.44it/s]


train: New cache created: /content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/data/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/data/valid/labels... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<00:00, 127.56it/s]


val: New cache created: /content/drive/MyDrive/Miniproject/Datasets/2024.04.11_미니프로젝트 4차_실습자료/data/valid/labels.cache
Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         40        304      0.679      0.754       0.73      0.503
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 37, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



57 epochs completed in 0.056 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 22.5MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.66it/s]


                   all         40        304      0.754      0.769      0.813      0.564
             cool roof         40         38      0.688      0.711      0.739      0.533
          generic roof         40        266       0.82      0.827      0.887      0.594
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78e403a17ac0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
import os

best_weights_path = '/content/runs/detect/train9/weights/best.pt'

if os.path.exists(best_weights_path):
    model = YOLO(best_weights_path)
    results = model.predict(source=data_path + 'test/*.png',
                            save=True,
                            conf=0.5,
                            iou=0.3,
                            line_width=2)
else:
    print(f"Error: '{best_weights_path}' 파일을 찾을 수 없습니다.")

Error: '/content/runs/detect/train9/weights/best.pt' 파일을 찾을 수 없습니다.


In [ ]:
pred_path = glob.glob('/content/runs/detect/predict3/*.png')

for path in pred_path:
    plt.imshow(image.load_img(path))
    plt.show()